In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

from FaaSEnv import FaaSEnv

In [2]:
env = FaaSEnv()

model = A2C(
    "MlpPolicy",
    env=env,
    verbose=0,
    tensorboard_log="./tensorboard/"
)

=> Loading checkpoint
=> Loading success


In [ ]:
model.learn(total_timesteps=1e6)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1, render=False)
env.close()
print(mean_reward, std_reward)

In [ ]:
real = []
ppo = []
lstm = []

obs = env.reset()
for _ in range(100):
    action, state = model.predict(observation=obs)
    ppo.append(action * 100)
    obs, reward, done, info = env.step(action)
    real.append(obs[0][7] * 100)
    lstm.append(obs[2][0] * 100)
    env.render()

from matplotlib import pyplot as plt

plt.plot(real, 'b', label='real')
plt.plot(lstm, 'r', label='lstm')
plt.plot(ppo, 'g', label='ppo')

In [ ]:
model.save("./model/a2c.pkl")